# Rainbow options with bruteforce methodology

In this Notebook we will go through the implementation using QMOD for the rainbow option.
This Notebook role is to verify result of different metodology on a smal scale problem, as it grows exponentially in the gate count.

### Introduction

In finance, a crucial aspect of asset pricing pertains to derivatives.
Derivatives are contracts whose value is contingent upon another source, known as the underlying.
The pricing of options, a specific derivative instrument, involves determining the fair market value (discounted payoff) of contracts affording their holders the right, though not the obligation, to buy (call) or sell (put) one or more underlying assets at a predefined strike price by a specified future expiration date (maturity date).
This process relies on mathematical models, considering variables like current asset prices, time to expiration, volatility, and interest rates.

## Data Definitions

The problem inputs are:
- `NUM_QUBITS`: the number of qubits representing an underlying asset
- `NUM_ASSETS`: the number of underlying assets
- `K`: the strike price
- `S0`: the arrays of underlying assets prices
- `dt`: the number of days to the maturity date
- `COV`: the covariance matrix that correlate the underlyings
- `MU_LOG_RET`: the array containing the mean of the log return of each underlyings

In [1]:
import numpy as np
import scipy

NUM_QUBITS = 2
NUM_ASSETS = 2

K = 190
S0 = [193.97, 189.12]
dt = 250

COV = np.array([[0.000335, 0.000257], [0.000257, 0.000418]])
MU_LOG_RET = np.array([0.00050963, 0.00062552])

In [2]:
MU = MU_LOG_RET * dt
CHOLESKY = np.linalg.cholesky(COV) * np.sqrt(dt)
SCALING_FACTOR = 1 / CHOLESKY[0, 0]

In [4]:
from classiq import *

EPSILON_VALUE = 0.05
ALPHA_VALUE = 0.1
EPSILON = QConstant("EPSILON", float, EPSILON_VALUE)
ALPHA = QConstant("ALPHA", float, ALPHA_VALUE)

## Gaussian State preparation
Encode the probability distribution of a discrete multivariate random variable $W$ taking values in $\{w_0, .., w_{N-1}\}$ describing the assets' prices at the maturity date. The number of discretized values, denoted as $N$, depends on the precision of the state preparation module and is consequently connected to the number of qubits ($n=$) according to the formula $N=2^n$. 

$$
\sum_{i=0}^{N-1} \sqrt{p(w_i)}\left|w_i\right\rangle 
$$

In [5]:
def gaussian_discretization(num_qubits, mu=0, sigma=1, stds_around_mean_to_include=3):
    lower = mu - stds_around_mean_to_include * sigma
    upper = mu + stds_around_mean_to_include * sigma
    num_of_bins = 2**num_qubits
    sample_points = np.linspace(lower, upper, num_of_bins + 1)

    def single_gaussian(x: np.ndarray, _mu: float, _sigma: float) -> np.ndarray:
        cdf = scipy.stats.norm.cdf(x, loc=_mu, scale=_sigma)
        return cdf[1:] - cdf[0:-1]

    non_normalized_pmf = (single_gaussian(sample_points, mu, sigma),)
    real_probs = non_normalized_pmf / np.sum(non_normalized_pmf)
    return sample_points[:-1], real_probs[0].tolist()


grid_points, probabilities = gaussian_discretization(NUM_QUBITS)

STEP_X = grid_points[1] - grid_points[0]
MIN_X = grid_points[0]

### SANITY CHECK

The process must be stopped if the strike price $K$ is greater than the maximum value reacheable by the assets during the simulation, to avoid meaningless results. The payoff is $0$ in this case, so there is no need to simulate.

In [6]:
from IPython.display import Markdown

if K >= max(S0 * np.exp(np.dot(CHOLESKY, [grid_points[-1]] * 2) + MU)):
    display(
        Markdown(
            "<font color='red'> K always greater than the maximum asset values. Stop the run, the payoff is 0</font>"
        )
    )

## Maximum Computation

### Precision utils

In [7]:
FRAC_PLACES = 1


def round_factor(a):
    precision_factor = 2 ** (FRAC_PLACES)
    return np.floor(a * precision_factor) / precision_factor


def floor_factor(a):
    precision_factor = 2 ** (FRAC_PLACES)
    return np.floor(a * precision_factor) / precision_factor

### Affine and maximum arithmetic definitions

Considering the time delta between the starting date ($t_0$) and the maturity date ($t$), we can express the return value $R_i$ for the $i$-th asset as $R_i = \mu_i + y_i$. Where:

  $\mu_i= (t-t_0)\tilde{\mu}_i$, being $\tilde{\mu}_i$ the expected daily log-return value. It can be estimated by considering the historical time series of log returns for the $i$-th asset.

$y_i$ is obtained through the dot product between the matrix $\mathbf{L}$ and the standard multivariate Gaussian sample:

$$    y_i = \Delta x \cdot \sum_kl_{ik}d_k + x_{min} \cdot \sum_k l_{ik} $$
$\Delta x$ is the Gaussian discretization step, $x_{min}$ is the lower Gaussian truncation value and $d_k \in [0,2^m-1]$ is the sample taken from the $k$-th standard Gaussian.
$l_{ik}$ is the $i,k$ entry of the matrix $\mathbf{L}$, defined as $\mathbf{L}=\mathbf{C}\sqrt{(t-t_0)}$, where $\mathbf{C}$ is the lower triangular matrix obtained by applying the Cholesky decomposition to the historical daily log-returns correlation matrix.

In [8]:
from functools import reduce

from classiq import *
from classiq.qmod.symbolic import max as qmax

a = STEP_X / SCALING_FACTOR
b = np.log(S0[0]) + MU[0] + MIN_X * CHOLESKY[0].sum()


def get_affine_formula(assets, i):
    return reduce(
        lambda x, y: x + y,
        [
            assets[j] * round_factor(SCALING_FACTOR * CHOLESKY[i, j])
            for j in range(NUM_ASSETS)
            if CHOLESKY[i, j]
        ],
    )


c = (
    SCALING_FACTOR
    * (
        np.log(S0[1])
        + MU[1]
        - (np.log(S0[0]) + MU[0])
        + MIN_X * sum(CHOLESKY[1] - CHOLESKY[0])
    )
    / (STEP_X)
)

c = round_factor(c)


def calculate_max_reg_type():
    x1 = QNum("x1", NUM_QUBITS, False, 0)
    x2 = QNum("x2", NUM_QUBITS, False, 0)
    expr = qmax(get_affine_formula([x1, x2], 0), get_affine_formula([x1, x2], 1) + c)
    size_in_bits, sign, fraction_digits = get_expression_numeric_attributes(
        [x1, x2], expr
    )
    return size_in_bits, fraction_digits


MAX_NUM_QUBITS = calculate_max_reg_type()[0]
MAX_FRAC_PLACES = calculate_max_reg_type()[1]

In [9]:
@qfunc
def affine_max(x1: QNum, x2: QNum, res: Output[QNum]):
    res |= qmax(get_affine_formula([x1, x2], 0), get_affine_formula([x1, x2], 1) + c)

## Brute-Force Amplitude Loading Method

This type of amplitude loading has an exponential scale, is therefore used as a "sanity check" method for validating result from the direct method and integration method that are part of the paper [[1]](#QALROP)

In [10]:
from classiq import *
from classiq.qmod.symbolic import exp, sqrt


def get_payoff_expression(x, size, fraction_digits):
    payoff = sqrt(
        qmax(
            S0[0]
            * exp(
                STEP_X / SCALING_FACTOR * (2 ** (size - fraction_digits)) * x
                + (MU[0] + MIN_X * CHOLESKY[0].sum())
            ),
            K,
        )
    )
    return payoff


def get_payoff_expression_normalized(x: QNum, size, fraction_digits):
    x_max = 1 - 1 / (2**size)
    payoff_max = get_payoff_expression(x_max, size, fraction_digits)
    payoff = get_payoff_expression(x, size, fraction_digits)
    return payoff / payoff_max


@qfunc
def brute_force_payoff(max_reg: QNum, ind_reg: QBit):
    max_reg_fixed = QNum("max_reg_fixed", max_reg.size, False, max_reg.size)
    bind(max_reg, max_reg_fixed)
    ind_reg *= get_payoff_expression_normalized(
        max_reg_fixed, max_reg.size, max_reg.fraction_digits
    )
    bind(max_reg_fixed, max_reg)

In [11]:
class EstimationVars(QStruct):
    x1: QNum[NUM_QUBITS, False, 0]
    x2: QNum[NUM_QUBITS, False, 0]
    ind: QBit


@qfunc
def rainbow_brute_force(qvars: EstimationVars) -> None:
    inplace_prepare_state(probabilities, 0, qvars.x1)
    inplace_prepare_state(probabilities, 0, qvars.x2)

    max_out = QNum("max_out")
    within_apply(
        lambda: affine_max(qvars.x1, qvars.x2, max_out),
        lambda: brute_force_payoff(max_out, qvars.ind),
    )


@qfunc
def main(qvars: Output[EstimationVars]) -> None:
    allocate(qvars.size, qvars)
    rainbow_brute_force(qvars)


constraints = Constraints(max_width=23)
qmod = create_model(main, constraints=constraints)
qprog = synthesize(qmod)
show(qprog)

Opening: http://localhost:4200/circuit/6d12486c-344e-45db-9bda-c31f52a55788?version=0.0.0


## IQAE algorithm

In [12]:
from classiq import Z, cfunc
from classiq.qmod.builtins.classical_execution_primitives import iqae, save


@qfunc
def qmci_oracle(qvars: EstimationVars):
    Z(qvars.ind)


@cfunc
def cmain():
    iqae_res = iqae(epsilon=EPSILON, alpha=ALPHA)
    save({"iqae_res": iqae_res})

In [13]:
from classiq import CInt, QArray, QCallable, grover_operator, power


@qfunc
def grover_algorithm(
    k: CInt,
    oracle_operand: QCallable[QArray[QBit]],
    sp_operand: QCallable[QArray[QBit]],
    x: QArray[QBit],
):
    sp_operand(x)
    power(k, lambda: grover_operator(oracle_operand, sp_operand, x))

In [14]:
def get_main():
    @qfunc
    def main(
        k: CInt,
        ind: Output[QBit],
    ) -> None:
        qvars = EstimationVars("qvars")
        allocate(qvars.size, qvars)
        grover_algorithm(
            k,
            qmci_oracle,
            rainbow_brute_force,
            qvars,
        )
        state_reg = QArray("state_reg")
        bind(qvars, [state_reg, ind])

    return main

In [15]:
from classiq import execute, write_qmod
from classiq.execution import ExecutionPreferences


def synthesize_and_execute(post_process):
    constraints = Constraints(max_width=25)
    qmod = create_model(
        get_main(),
        constraints=constraints,
        classical_execution_function=cmain,
    )
    write_qmod(qmod, "rainbow_options_bruteforce_method")
    print("Starting synthesis")
    qprog = synthesize(qmod)
    show(qprog)
    print("Starting execution")
    res = execute(qprog).result()
    iqae_res = res[0].value
    parsed_res = post_process(res[0].value)

    return (qmod, qprog, iqae_res, parsed_res)

## Post Process
We need to add to the post-processing function a term:
\begin{equation}
\begin{split}
&\mathbb{E} \left[\max\left(e^{b \cdot z}, Ke^{-b'}\right) \right] e^{b'} - K \\
= &\mathbb{E} \left[\max\left(e^{-a\hat{x}}, Ke^{-b'-ax_{max}}\right) \right]e^{b'+ ax_{max}} - K 
\end{split}
\end{equation}

In [16]:
import sympy

payoff_expression = f"sqrt(max({S0[0]} * exp({STEP_X / SCALING_FACTOR * (2 ** (MAX_NUM_QUBITS - MAX_FRAC_PLACES))} * x + ({MU[0]+MIN_X*CHOLESKY[0].sum()})), {K}))"
payoff_func = sympy.lambdify(sympy.symbols("x"), payoff_expression)
payoff_max = payoff_func(1 - 1 / (2**MAX_NUM_QUBITS))


def parse_result_bruteforce(iqae_res):
    option_value = iqae_res.estimation * (payoff_max**2) - K
    confidence_interval = np.array(iqae_res.confidence_interval) * (payoff_max**2) - K
    return (option_value, confidence_interval)

# Run Method

In [17]:
qmod_brute_force, qprog_brute_force, iqae_res_brute_force, parsed_res_brute_force = (
    synthesize_and_execute(parse_result_bruteforce)
)
result, conf_interval = parsed_res_brute_force
print(
    f"raw iqae results: {iqae_res_brute_force.estimation} with confidence interval {iqae_res_brute_force.confidence_interval}"
)
print(f"option estimated value: {result} with confidence interval {conf_interval}")

Starting synthesis
Opening: http://localhost:4200/circuit/345928f7-fef8-463e-83d9-576a2cc9a09d?version=0.0.0
Starting execution
raw iqae results: 0.513 with confidence interval (0.4677543120163805, 0.5582456879836195)
option estimated value: 26.756022317711 with confidence interval [ 7.6385265  45.87351814]


In [18]:
expected_payoff = 23.0238
ALPHA_ASSERTION = 1e-5
measured_confidence = conf_interval[1] - conf_interval[0]
confidence_scale_by_alpha = np.sqrt(
    np.log(ALPHA_VALUE / ALPHA_ASSERTION)
)  # based on e^2=(1/2N)*log(2T/alpha) from "Iterative Quantum Amplitude Estimation" since our alpha is low, we want to check within a bigger confidence interval
assert (
    np.abs(result - expected_payoff)
    <= 0.5 * measured_confidence * confidence_scale_by_alpha
), f"Payoff result is out of the {ALPHA_ASSERTION*100}% confidence interval: |{result} - {expected_payoff}| > {0.5*measured_confidence * confidence_scale_by_alpha}"

## References

<a name='QALROP'>[1]</a>: [Francesca Cibrario et al., Quantum Amplitude Loading for Rainbow Options Pricing. Preprint](https://arxiv.org/abs/2402.05574v2)
